In [ ]:
# 是否已整理过数据集
org = True
if not org:
    import os
    import shutil

    def reorg_cifar10_data(data_dir, label_file, train_dir, test_dir, input_dir, valid_ratio):
        with open(os.path.join(data_dir, label_file), 'r') as f:
            lines = f.readlines()[1:]
            tokens = [l.rstrip().split(',') for l in lines]
            idx_label = dict(((int(idx), label) for idx, label in tokens))
        labels = set(idx_label.values())
        num_train = len(os.listdir(os.path.join(data_dir, train_dir)))
        num_train_tuning = int(num_train * (1 - valid_ratio))
        assert 0 < num_train_tuning < num_train
        num_train_tuning_per_label = num_train_tuning // len(labels)
        label_count = dict()
        def mkdir_if_not_exist(path):
            if not os.path.exists(os.path.join(*path)):
                os.makedirs(os.path.join(*path))
        for train_file in os.listdir(os.path.join(data_dir, train_dir)):
            idx = int(train_file.split('.')[0])
            label = idx_label[idx]
            mkdir_if_not_exist([data_dir, input_dir, 'train_valid', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),os.path.join(data_dir, input_dir, 'train_valid', label))
            if label not in label_count or label_count[label] < num_train_tuning_per_label:
                mkdir_if_not_exist([data_dir, input_dir, 'train', label])
                shutil.copy(os.path.join(data_dir, train_dir, train_file),os.path.join(data_dir, input_dir, 'train', label))
                label_count[label] = label_count.get(label, 0) + 1
            else:
                mkdir_if_not_exist([data_dir, input_dir, 'valid', label])
                shutil.copy(os.path.join(data_dir, train_dir, train_file),os.path.join(data_dir, input_dir, 'valid', label))
        mkdir_if_not_exist([data_dir, input_dir, 'test', 'unknown'])
        for test_file in os.listdir(os.path.join(data_dir, test_dir)):
            shutil.copy(os.path.join(data_dir, test_dir, test_file),os.path.join(data_dir, input_dir, 'test', 'unknown'))

    train_dir = 'train'
    test_dir = 'test'

    data_dir = '/home/sinyer/python/data/kaggle_cifar10'
    label_file = 'trainLabels.csv'
    input_dir = 'train_valid_test'
    valid_ratio = 0.1
    reorg_cifar10_data(data_dir, label_file, train_dir, test_dir, input_dir, valid_ratio)

In [1]:
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision
import numpy as np
from mxnet.gluon import nn
import mxnet as mx
from time import time
import matplotlib.pyplot as plt
ctx = mx.gpu()

data_dir = '/home/sinyer/python/data/kaggle_cifar10'
label_file = 'trainLabels.csv'
input_dir = 'train_valid_test'

input_str = data_dir + '/' + input_dir + '/'

train_valid_ds = vision.ImageFolderDataset(input_str + 'train_valid', flag=1)

train_pic = nd.stack(*[train_valid_ds.__getitem__(i)[0] for i in range(50000)])
train_label = []
for i in range(50000):
    train_label.append(train_valid_ds.__getitem__(i)[1]) 
train_label = nd.array(train_label)

train_pic = train_pic.astype('float32')/255
train_label = train_label.astype('float32')

batch_size = 128
mean = [0.4914, 0.4822, 0.4465]
std = [0.2470, 0.2435, 0.2616]
for i in range(3):
    train_pic[:,:,:,i] = (train_pic[:,:,:,i] - mean[i])/std[i]
train_pic = np.transpose(train_pic, (0,3,1,2))

train_data = gluon.data.DataLoader(gluon.data.ArrayDataset(train_pic, train_label), batch_size, shuffle=True)

In [2]:
def accuracy(output, label):
    return nd.mean(output.argmax(axis=1)==label).asscalar()

def apply(img):
    aug_train = image.CreateAugmenter(data_shape=(3, 32, 32), rand_crop=True, rand_mirror=True)
    for aug in aug_train:
        img = aug(img)
    return img

def tf(data): 
    data = nd.pad(data, pad_width=(0,0,0,0,2,2,2,2),mode='constant',constant_value=0)
    data = nd.transpose(data, (0,2,3,1))
    data = nd.stack(*[apply(d) for d in data])
    data = nd.transpose(data, (0,3,1,2))
    return data

class Residual(nn.Block):
    def __init__(self, channels, same_shape=True, equal=True, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.same_shape = same_shape
        self.equal = equal
        with self.name_scope():
            strides = 1 if same_shape else 2
            self.bn1 = nn.BatchNorm()
            self.conv1 = nn.Conv2D(channels, kernel_size=3, padding=1, strides=strides)
            self.bn2 = nn.BatchNorm()
            self.conv2 = nn.Conv2D(channels, kernel_size=3, padding=1)
            if (not same_shape) or (not equal):
                self.conv3 = nn.Conv2D(channels, kernel_size=1, strides=strides)
    def forward(self, x):
        out = self.conv1(nd.relu(self.bn1(x)))
        out = self.conv2(nd.relu(self.bn2(out)))
        if (not self.same_shape) or (not self.equal):
            x = self.conv3(x)
        return out + x

class ResNet(nn.Block):
    def __init__(self, num_classes, **kwargs):
        super(ResNet, self).__init__(**kwargs)
        with self.name_scope(): 
            net = self.net = nn.Sequential()
            net.add(nn.Conv2D(channels=16, kernel_size=3, strides=1, padding=1))
            net.add(Residual(channels=16*8, equal=False))
            net.add(Residual(channels=16*8), Residual(channels=16*8))            
            net.add(Residual(channels=32*8, same_shape=False))
            net.add(Residual(channels=32*8), Residual(channels=32*8))
            net.add(Residual(channels=64*8, same_shape=False))
            net.add(Residual(channels=64*8), Residual(channels=64*8))
            net.add(nn.BatchNorm())
            net.add(nn.Activation(activation='relu'))
            net.add(nn.AvgPool2D(pool_size=8))
            net.add(nn.Flatten())
            net.add(nn.Dense(num_classes))
    def forward(self, x):
        out = x
        for i, b in enumerate(self.net):
            out = b(out)
        return out

In [3]:
net = ResNet(10)
net.initialize(ctx=ctx, init=init.Xavier())
loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'nag', {'learning_rate': 0.1, 'momentum': 0.9, 'wd': 5e-4})

In [4]:
epochs = 160

for epoch in range(epochs):
    if epoch == 60:
        trainer.set_learning_rate(0.02)
    if epoch == 120:
        trainer.set_learning_rate(0.004)
    if epoch == 140:
        trainer.set_learning_rate(0.0008)
    train_loss = 0.
    train_acc = 0.
    batch = 0
    start = time()
    for data, label in train_data:
        data = tf(data).as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            l = loss(output, label)
        l.backward()
        trainer.step(data.shape[0])
        train_loss += nd.mean(l).asscalar()
        train_acc += accuracy(output, label)
        batch += 1
    if epoch%1 == 0:
        print("E %d; L %f; Tr_acc %f; T %f" % (epoch, train_loss / batch, train_acc / batch, time() - start))
print("Tr_acc %f" % (train_acc / batch))

E 0; L 1.524124; Tr_acc 0.441448; T 122.589915
E 1; L 0.965602; Tr_acc 0.655790; T 119.494456
E 2; L 0.744971; Tr_acc 0.740181; T 119.443863
E 3; L 0.621808; Tr_acc 0.784871; T 118.745010
E 4; L 0.544807; Tr_acc 0.812080; T 119.453280
E 5; L 0.505423; Tr_acc 0.825464; T 118.908199
E 6; L 0.468818; Tr_acc 0.839726; T 118.610433
E 7; L 0.447672; Tr_acc 0.845033; T 118.821814
E 8; L 0.433558; Tr_acc 0.851578; T 119.065236
E 9; L 0.417138; Tr_acc 0.856981; T 118.695992
E 10; L 0.401701; Tr_acc 0.862112; T 118.618255
E 11; L 0.385744; Tr_acc 0.867479; T 118.668333
E 12; L 0.380615; Tr_acc 0.868962; T 118.877657
E 13; L 0.371043; Tr_acc 0.873537; T 118.883992
E 14; L 0.361567; Tr_acc 0.876586; T 118.941867
E 15; L 0.355193; Tr_acc 0.879168; T 118.572304
E 16; L 0.343887; Tr_acc 0.881989; T 118.750083
E 17; L 0.334025; Tr_acc 0.884195; T 118.573804
E 18; L 0.332493; Tr_acc 0.885546; T 118.413831
E 19; L 0.331624; Tr_acc 0.886881; T 118.142120
E 20; L 0.326648; Tr_acc 0.887960; T 118.424217
E 

In [5]:
test_ds = vision.ImageFolderDataset(input_str + 'test', flag=1)

test_pic = nd.stack(*[test_ds.__getitem__(i)[0] for i in range(300000)])
test_label = []
for i in range(300000):
    test_label.append(test_ds.__getitem__(i)[1]) 
test_label = nd.array(test_label)

test_pic = test_pic.astype('float32')/255
test_label = test_label.astype('float32')

for i in range(3):
    test_pic[:,:,:,i] = (test_pic[:,:,:,i] - mean[i])/std[i]    
test_pic = np.transpose(test_pic, (0,3,1,2))

test_data = gluon.data.DataLoader(gluon.data.ArrayDataset(test_pic, test_label), batch_size, shuffle=False)

In [6]:
import numpy as np
import pandas as pd

preds = []
for data, label in test_data:
    output = net(data.as_in_context(ctx))
    preds.extend(output.argmax(axis=1).astype(int).asnumpy())

sorted_ids = list(range(1, len(test_ds) + 1))
sorted_ids.sort(key = lambda x:str(x))

df = pd.DataFrame({'id': sorted_ids, 'label': preds})
df['label'] = df['label'].apply(lambda x: train_valid_ds.synsets[x])
df.to_csv('submission.csv', index=False)